In [68]:
import cv2
import sys
import os
import dlib
import glob
import csv
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
%matplotlib inline

In [83]:
images = []
labels = []
resize_x = 80
resize_y = 80

In [84]:
def conv2d(x, W):
    return tf.nn.conv2d(
        x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [85]:
with open('../data/convolutional/data.csv', 'rb') as mycsvfile:
    thedata = csv.reader(mycsvfile, delimiter=':')
    thedata.next()
    for row in thedata:
        images.append(np.fromstring(row[0].replace("[", "").replace("]", ""), dtype=int, sep=" "))
        labels.append(row[1])

In [86]:
labels = np.array(labels)

In [87]:
df_labels = pd.get_dummies(labels)
labels = np.array(df_labels)

In [88]:
images = np.array(images) / 255

In [89]:
X_train = images[:400]
Y_train = labels.reshape(len(labels), -1)[:400]

X_test = images[20:]
Y_test = labels.reshape(len(labels), -1)[20:]

In [96]:
Y_test

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,

In [91]:
# Input and output variables

INPUTS = resize_x * resize_y * 3
OUTPUTS = 2
BATCH_SIZE = 20
NUM_EPOCHS = 51
LEARNING_RATE = 1e-04

In [92]:
# Interactive session because of notebook environment
sess = tf.InteractiveSession()

# Input and output placeholder
x = tf.placeholder(tf.float32, [None, INPUTS])
y = tf.placeholder(tf.float32, [None, OUTPUTS])

# First Convolutional Layer
# Input and output placeholder
x = tf.placeholder(tf.float32, [None, INPUTS])
y = tf.placeholder(tf.float32, [None, OUTPUTS])
pkeep = tf.placeholder(tf.float32)

# First Convolutional Layer
x_image = tf.reshape(x, [-1, resize_x, resize_y, 3])
W_conv_1 = tf.Variable(tf.truncated_normal([2, 2, 3, 8], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.0, shape=[8]))
h_conv_1 = tf.nn.relu(conv2d(x_image, W_conv_1) + b_conv_1)
h_pool_1 = max_pool_2x2(h_conv_1)

# Second Convolutional Layer
W_conv_2 = tf.Variable(tf.truncated_normal([2, 2, 8, 32], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.0, shape=[32]))
h_conv_2 = tf.nn.relu(conv2d(h_pool_1, W_conv_2) + b_conv_2)
h_pool_2 = max_pool_2x2(h_conv_2)

# Third Convolutional Layer
W_conv_3 = tf.Variable(tf.truncated_normal([2, 2, 32, 128], stddev=0.1))
b_conv_3 = tf.Variable(tf.constant(0.0, shape=[128]))
h_conv_3 = tf.nn.relu(conv2d(h_pool_2, W_conv_3) + b_conv_3)
h_pool_3 = max_pool_2x2(h_conv_3)

# Fourth Convolutional Layer
W_conv_4 = tf.Variable(tf.truncated_normal([2, 2, 128, 256], stddev=0.1))
b_conv_4 = tf.Variable(tf.constant(0.0, shape=[256]))
h_conv_4 = tf.nn.relu(conv2d(h_pool_3, W_conv_4) + b_conv_4)
h_pool_4 = max_pool_2x2(h_conv_4)

# Densely connected layer
W_fc1 = tf.Variable(tf.truncated_normal([5 * 5 * 256, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.0, shape=[1024]))
h_poolfc1_flat = tf.reshape(h_pool_4, [-1, 5 * 5 * 256])
h_fc1 = tf.nn.relu(tf.matmul(h_poolfc1_flat, W_fc1) + b_fc1)

# Densely connected layer
W_fc2 = tf.Variable(tf.truncated_normal([1024, 512], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.0, shape=[512]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

# Dropout
h_drop = tf.nn.dropout(h_fc2, pkeep)

# Read out Layer
W_fc3 = tf.Variable(tf.truncated_normal([512, OUTPUTS], stddev=0.1))
b_fc3 = tf.Variable(tf.constant(0.0, shape=[OUTPUTS]))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf.matmul(h_drop, W_fc3) + b_fc3, y))
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train_step = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(tf.matmul(h_drop, W_fc3) + b_fc3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess.run(init)

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f95afe59050>> ignored


In [93]:
loss_train_array = []
test_accuracy_array = []
train_accuracy_array = []

for current_epoch in range(NUM_EPOCHS):
    shuffled_index = np.arange(X_train.shape[0])
    np.random.shuffle(shuffled_index)

    train_dataset = X_train[shuffled_index]
    train_labels = Y_train[shuffled_index]

    for step in xrange(int(X_train.shape[0] / BATCH_SIZE)):
        
        offset = step * BATCH_SIZE
        batch_data = train_dataset[offset:(offset + BATCH_SIZE)]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {x: batch_data, y: batch_labels, pkeep: 0.8}
        _, loss_train = sess.run([train_step, loss],
                                 feed_dict=feed_dict)

    # We calculate the accuracies to plot their values later
    loss_train_array.append(loss_train)
    
    train_accuracy = sess.run(
        accuracy, feed_dict={x: X_train, y: Y_train, pkeep: 1.0})
    
    train_accuracy_array.append(train_accuracy)
    
    test_accuracy = sess.run(
        accuracy, feed_dict={x: X_test, y: Y_test, pkeep: 1.0})
    
    test_accuracy_array.append(test_accuracy)

    print (
        'Epoch %04d, '
        'loss train %.8f, train accuracy %.8f, test accuracy %.8f'
        %
        (current_epoch,
         loss_train,
         train_accuracy,
         test_accuracy))

Epoch 0000, loss train 0.01551567, train accuracy 0.99250001, test accuracy 0.99255586
Epoch 0001, loss train 0.01542919, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0002, loss train 0.00386496, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0003, loss train 0.00334567, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0004, loss train 0.00110674, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0005, loss train 0.00354418, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0006, loss train 0.00051712, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0007, loss train 0.00133888, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0008, loss train 0.00025599, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0009, loss train 0.00007332, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0010, loss train 0.00012525, train accuracy 1.00000000, test accuracy 1.00000000
Epoch 0011, loss train 0.00033774, train ac

In [94]:
# Comparing values
y_ = tf.matmul(h_drop, W_fc3) + b_fc3
predicted = y_.eval(feed_dict={x: X_test, pkeep: 0.8})
predicted

array([[  6.553792  ,  -5.43515158],
       [  8.54269218,  -7.19680262],
       [  6.24348688,  -6.14228439],
       [  7.8774662 ,  -6.42351675],
       [  6.56518793,  -6.39955282],
       [  8.07974148,  -6.77744246],
       [  7.61159945,  -5.78816175],
       [  7.2906909 ,  -5.00560427],
       [  6.31061077,  -6.76134014],
       [  8.91503525,  -5.64068174],
       [  6.99686384,  -5.09020233],
       [  6.76495981,  -4.63457489],
       [  8.04547882,  -5.44517422],
       [  6.4825387 ,  -5.06725168],
       [  8.5867033 ,  -4.27741289],
       [  7.28539801,  -7.01778793],
       [  7.26175737,  -6.5471487 ],
       [  7.63127279,  -5.67450094],
       [  7.20974684,  -5.19469309],
       [  8.18650723,  -6.55686235],
       [  7.12688684,  -6.44904947],
       [  8.11149788,  -5.30140972],
       [  8.47350216,  -6.46562052],
       [  8.41492367,  -5.72136354],
       [  8.01280785,  -6.25949907],
       [  7.33635187,  -6.71458197],
       [  6.49941015,  -6.73085833],
 